> ### EEE2020: Data Structures & Algorithms

# Lecture 10: Binary Tree Applications

## 1. Parse Tree

In [1]:
from stack import Stack

In [2]:
class BinaryTree:
    def __init__(self,rootObj):
        self.key = rootObj
        self.leftChild = None
        self.rightChild = None

    def insertLeft(self,newNode):
        if self.leftChild == None:
            self.leftChild = BinaryTree(newNode)
        else:
            t = BinaryTree(newNode)
            t.leftChild = self.leftChild
            self.leftChild = t

    def insertRight(self,newNode):
        if self.rightChild == None:
            self.rightChild = BinaryTree(newNode)
        else:
            t = BinaryTree(newNode)
            t.rightChild = self.rightChild
            self.rightChild = t


    def getRightChild(self):
        return self.rightChild

    def getLeftChild(self):
        return self.leftChild

    def setRootVal(self,obj):
        self.key = obj

    def getRootVal(self):
        return self.key

In [3]:
def buildParseTree(fpexp):
    fplist = fpexp.split()
    pStack = Stack()
    eTree = BinaryTree('')
    pStack.push(eTree)
    currentTree = eTree

    for i in fplist: #fplist에 들어있는것 만큼 반복함
        if i == '(': #currentTree의 left child를 만들고 거기로 이동, currentTree는 pstack으로
            currentTree.insertLeft('')
            pStack.push(currentTree)
            currentTree = currentTree.getLeftChild()

        elif i not in ['+', '-', '*', '/', ')']: #숫자가 들어오면
            currentTree.setRootVal(int(i)) #값을 root로 설정
            parent = pStack.pop() 
            currentTree = parent #parent node로 복귀
            
        elif i in ['+', '-', '*', '/']:
            currentTree.setRootVal(i) #rootval로 설정
            currentTree.insertRight('') #right_child생성
            pStack.push(currentTree) #지금 노드를 pStack으로 
            currentTree = currentTree.getRightChild() #다음 노드는 right_child

        elif i == ')': #괄호가 닫히면 그 위 노드로 올라가든지 함
            currentTree = pStack.pop()

        else:
            raise ValueError("token '{}' is not a valid integer".format(i))

    return eTree

In [4]:
import operator
def evaluate(parseTree):
    opers = {'+':operator.add, '-':operator.sub, '*':operator.mul, '/':operator.truediv}

    leftC = parseTree.getLeftChild()
    rightC = parseTree.getRightChild()

    if leftC and rightC:
        fn = opers[parseTree.getRootVal()]
        return fn(evaluate(leftC),evaluate(rightC))
    else:
        return parseTree.getRootVal()

In [5]:
def printexp(tree):
    sVal = ""
    if tree:
        sVal = '(' + printexp(tree.getLeftChild())
        sVal = sVal + str(tree.getRootVal())
        sVal = sVal + printexp(tree.getRightChild())+')'
    return sVal

In [6]:
pt = buildParseTree("( 3 + ( 4 * 5 ) )")

In [7]:
evaluate(pt)

23

In [8]:
printexp(pt)

'((3)+((4)*(5)))'

In [9]:
pt = buildParseTree("( ( 10 + 5 ) * 3 )")

In [10]:
evaluate(pt)

45

In [11]:
printexp(pt)

'(((10)+(5))*(3))'

## 2. Tree Traversals

In [ ]:
def preorder(tree):
    if tree != None:
        print(tree.getRootVal())
        preorder(tree.getLeftChild())
        preorder(tree.getRightChild())

In [ ]:
def postorder(tree):
    if tree != None:
        postorder(tree.getLeftChild())
        postorder(tree.getRightChild())
        print(tree.getRootVal())

In [ ]:
def inorder(tree):
    if tree != None:
        inorder(tree.getLeftChild())
        print(tree.getRootVal())
        inorder(tree.getRightChild())

In [ ]:
t = BinaryTree(7)
t.insertLeft(3)
t.insertRight(9)

In [ ]:
preorder(t)

In [ ]:
inorder(t)

In [ ]:
postorder(t)

In [ ]:
T = BinaryTree('-')
T.insertLeft('/')
T.insertRight('+')
T.getLeftChild().insertLeft('X')
T.getLeftChild().insertRight('+')
T.getRightChild().insertLeft('X')
T.getRightChild().insertRight('6')

In [ ]:
preorder(T)

In [ ]:
inorder(T)

In [ ]:
postorder(T)

## 2. Tree Traversals: Binary Search Tree

In [4]:
class TreeNode:
    def __init__(self,key,val,left=None,right=None,parent=None):
        self.key = key
        self.payload = val
        self.leftChild = left
        self.rightChild = right
        self.parent = parent
        self.balanceFactor = 0 #이게 9번에서의 코드아 차이/밑에 avltree만들라고
        
    def hasLeftChild(self):
        return self.leftChild

    def hasRightChild(self):
        return self.rightChild
    
    def isLeftChild(self):
        return self.parent and self.parent.leftChild == self

    def isRightChild(self):
        return self.parent and self.parent.rightChild == self

    def isRoot(self):
        return not self.parent

    def isLeaf(self):
        return not (self.rightChild or self.leftChild)

    def hasAnyChildren(self):
        return self.rightChild or self.leftChild

    def hasBothChildren(self):
        return self.rightChild and self.leftChild
    
    def replaceNodeData(self,key,value,lc,rc):
        self.key = key
        self.payload = value
        self.leftChild = lc
        self.rightChild = rc
        if self.hasLeftChild():
            self.leftChild.parent = self
        if self.hasRightChild():
            self.rightChild.parent = self
        
    def findSuccessor(self):
        succ = None
        if self.hasRightChild(): #right_child가 있으면 successor이렇게 찾음
            succ = self.rightChild.findMin()
        else:
            if self.parent:
                if self.isLeftChild(): #parent가 있고 left_child면
                    succ = self.parent #succ은 parent로
                else:#parent가 있고 right_child면
                    self.parent.rightChild = None #parent의 right_child=None으로 만들고 
                    succ = self.parent.findSuccessor() #parent에서 findSuccessor찾음
                    self.parent.rightChild = self #다시 parent.right_child붙여줌
        return succ


    def spliceOut(self):
        if self.isLeaf():
            if self.isLeftChild():
                self.parent.leftChild = None
            else:
                self.parent.rightChild = None
        elif self.hasAnyChildren():
            if self.hasLeftChild():
                if self.isLeftChild():
                    self.parent.leftChild = self.leftChild
                else:
                    self.parent.rightChild = self.leftChild
                self.leftChild.parent = self.parent
            else:
                if self.isLeftChild():
                    self.parent.leftChild = self.rightChild
                else:
                    self.parent.rightChild = self.rightChild
                self.rightChild.parent = self.parent

    def findMin(self):
        current = self
        while current.hasLeftChild():
            current = current.leftChild
        return current

#     def __iter__(self):
#         """The standard inorder traversal of a binary tree."""
#         if self:
#             if self.hasLeftChild():
#                 for elem in self.leftChild:
#                     yield elem
#             yield self.key
#             if self.hasRightChild():
#                 for elem in self.rightChild:
#                     yield elem

    def __iter__(self):
        if self:
            if self.hasLeftChild():
                yield from self.leftChild
            yield self.key
            if self.hasRightChild():
                yield from self.rightChild

In [5]:
class BinarySearchTree:
    
    def __init__(self):
        self.root = None
        self.size = 0
    
    def put(self,key,val):
        if self.root:
            self._put(key,val,self.root)
        else:
            self.root = TreeNode(key,val)
        self.size = self.size + 1

    def _put(self,key,val,currentNode):
        if key < currentNode.key:
            if currentNode.hasLeftChild():
                self._put(key,val,currentNode.leftChild)
            else:
                currentNode.leftChild = TreeNode(key,val,parent=currentNode)
        else:
            if currentNode.hasRightChild():
                self._put(key,val,currentNode.rightChild)
            else:
                currentNode.rightChild = TreeNode(key,val,parent=currentNode)
            
    def __setitem__(self,k,v):
        self.put(k,v)

    def get(self,key):
        if self.root:
            res = self._get(key,self.root)
            if res:
                return res.payload
            else:
                return None
        else:
            return None
        
    def _get(self,key,currentNode):
        if not currentNode:
            return None
        elif currentNode.key == key:
            return currentNode
        elif key < currentNode.key:
            return self._get(key,currentNode.leftChild) #재귀
        else:
            return self._get(key,currentNode.rightChild) #재귀
            
        
    def __getitem__(self,key):
        res = self.get(key)
        if res:
            return res
        else:
            raise KeyError('Error, key not in tree')
            

    def __contains__(self,key):
        if self._get(key,self.root):
            return True
        else:
            return False
        
    def length(self):
        return self.size

    def __len__(self):
        return self.size

    def __iter__(self):
        return self.root.__iter__()
    
    def delete(self,key):
        if self.size > 1:
            nodeToRemove = self._get(key,self.root)
            if nodeToRemove:
                self.remove(nodeToRemove)
                self.size = self.size-1
            else:
                raise KeyError('Error, key not in tree')
        elif self.size == 1 and self.root.key == key:
            self.root = None
            self.size = self.size - 1
        else:
            raise KeyError('Error, key not in tree')

    def __delitem__(self,key):
        self.delete(key)
    
    def remove(self,currentNode):
        if currentNode.isLeaf(): #leaf
            if currentNode == currentNode.parent.leftChild:
                currentNode.parent.leftChild = None
            else:
                currentNode.parent.rightChild = None
        elif currentNode.hasBothChildren(): #interior
            succ = currentNode.findSuccessor()
            succ.spliceOut() #succ는 꼬리노드
            currentNode.key = succ.key
            currentNode.payload = succ.payload
        else: # this node has one child
            if currentNode.hasLeftChild():
                if currentNode.isLeftChild():
                    currentNode.leftChild.parent = currentNode.parent
                    currentNode.parent.leftChild = currentNode.leftChild
                elif currentNode.isRightChild():
                    currentNode.leftChild.parent = currentNode.parent
                    currentNode.parent.rightChild = currentNode.leftChild
                else:#root이고 한쪽child만 가지고 있는 경우->그냥 child_node로 replace한다.
                    currentNode.replaceNodeData(currentNode.leftChild.key,
                                       currentNode.leftChild.payload,
                                       currentNode.leftChild.leftChild,
                                       currentNode.leftChild.rightChild)
            else:
                if currentNode.isLeftChild():
                    currentNode.rightChild.parent = currentNode.parent
                    currentNode.parent.leftChild = currentNode.rightChild
                elif currentNode.isRightChild():
                    currentNode.rightChild.parent = currentNode.parent
                    currentNode.parent.rightChild = currentNode.rightChild
                else:
                    currentNode.replaceNodeData(currentNode.rightChild.key,
                                       currentNode.rightChild.payload,
                                       currentNode.rightChild.leftChild,
                                       currentNode.rightChild.rightChild)

In [6]:
my_tree = BinarySearchTree()

In [7]:
my_tree[17] = "a"
my_tree[35] = "c"
my_tree[2] = "d"
my_tree[11] = "e"
my_tree[29] = "f"
my_tree[38] = "g"
my_tree[9] = "h"
my_tree[16] = "i"
my_tree[7] = "j"
my_tree[8] = "k"

In [8]:
#        17 
#    |       |
#    7       35
#  |  |     |  |
#  2  11    29 38
#     | |
#     9 16
#     |
#     8

In [9]:
for key in my_tree:
    print(key, my_tree[key])

2 d
7 j
8 k
9 h
11 e
16 i
17 a
29 f
35 c
38 g


## 3. Generators and Interators

In [1]:
def factors(n): 
    results = []
    for k in range (1, n+1): 
        if n % k == 0:
            results.append(k)
    return results

In [2]:
a = factors(100)

In [3]:
a

[1, 2, 4, 5, 10, 20, 25, 50, 100]

In [4]:
def factors(n): 
    results = []
    for k in range (1, n+1): 
        if n % k == 0:
            yield k

In [5]:
a = factors(100)

In [6]:
next(a)

1

In [7]:
next(a)

2

In [8]:
next(a)

4

In [9]:
for factor in factors(100):
    print(factor) 

1
2
4
5
10
20
25
50
100


In [10]:
data = [1,2,4,8]

In [11]:
next(data)

TypeError: 'list' object is not an iterator

In [12]:
i = iter(data)

In [13]:
next(i)

1

In [14]:
next(i)

2

In [15]:
next(i)

4

In [16]:
next(i)

8

In [17]:
next(i)

StopIteration: 

## 4. AVL Tree

In [ ]:
class AVLTree(BinarySearchTree):

    def _put(self,key,val,currentNode):
        if key < currentNode.key:
            if currentNode.hasLeftChild():
                self._put(key,val,currentNode.leftChild)
            else:
                currentNode.leftChild = TreeNode(key,val,parent=currentNode)
                self.updateBalance(currentNode.leftChild)
        else:
            if currentNode.hasRightChild():
                self._put(key,val,currentNode.rightChild)
            else:
                currentNode.rightChild = TreeNode(key,val,parent=currentNode)
                self.updateBalance(currentNode.rightChild)                

    def updateBalance(self,node):
        if node.balanceFactor > 1 or node.balanceFactor < -1:
            self.rebalance(node)
            return
        if node.parent != None:
            if node.isLeftChild():
                node.parent.balanceFactor += 1
            elif node.isRightChild():
                node.parent.balanceFactor -= 1

            if node.parent.balanceFactor != 0:
                self.updateBalance(node.parent)

    def rebalance(self,node):
        if node.balanceFactor < 0:
            if node.rightChild.balanceFactor > 0:
                # Do an LR Rotation
                self.rotateRight(node.rightChild)
                self.rotateLeft(node)
            else:
                # single left
                self.rotateLeft(node)
        elif node.balanceFactor > 0:
            if node.leftChild.balanceFactor < 0:
                # Do an RL Rotation
                self.rotateLeft(node.leftChild)
                self.rotateRight(node)
            else:
                # single right
                self.rotateRight(node)

    def rotateLeft(self,rotRoot):
        newRoot = rotRoot.rightChild
        rotRoot.rightChild = newRoot.leftChild
        if newRoot.leftChild != None:
            newRoot.leftChild.parent = rotRoot
        newRoot.parent = rotRoot.parent
        if rotRoot.isRoot():
            self.root = newRoot
        else:
            if rotRoot.isLeftChild():
                rotRoot.parent.leftChild = newRoot
            else:
                rotRoot.parent.rightChild = newRoot
        newRoot.leftChild = rotRoot
        rotRoot.parent = newRoot
        rotRoot.balanceFactor = rotRoot.balanceFactor + 1 - min(newRoot.balanceFactor, 0)
        newRoot.balanceFactor = newRoot.balanceFactor + 1 + max(rotRoot.balanceFactor, 0)


    def rotateRight(self,rotRoot):
        newRoot = rotRoot.leftChild
        rotRoot.leftChild = newRoot.rightChild
        if newRoot.rightChild != None:
            newRoot.rightChild.parent = rotRoot
        newRoot.parent = rotRoot.parent
        if rotRoot.isRoot():
            self.root = newRoot
        else:
            if rotRoot.isRightChild():
                rotRoot.parent.rightChild = newRoot
            else:
                rotRoot.parent.leftChild = newRoot
        newRoot.rightChild = rotRoot
        rotRoot.parent = newRoot
        rotRoot.balanceFactor = rotRoot.balanceFactor - 1 - max(newRoot.balanceFactor, 0)
        newRoot.balanceFactor = newRoot.balanceFactor - 1 + min(rotRoot.balanceFactor, 0)

In [ ]:
bst = AVLTree()

In [ ]:
bst.put(40,'a')
bst.put(30,'b')
bst.put(50,'c')
bst.put(10,'d')
bst.put(35,'e')
bst.put(37,'f')

In [ ]:
for key in bst:
    print(key, bst[key])